<a href="https://colab.research.google.com/github/ahmadhajmosa/aicongress2018/blob/master/Gemran_Lv1_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and upload Spacy models

In [0]:
!python -m spacy download de_core_news_md

     |████████████████████████████████| 224.6MB 1.2MB/s 
  Created wheel for de-core-news-md: filename=de_core_news_md-2.2.5-cp36-none-any.whl size=228399478 sha256=d507a6ea26d4da15570db212c5e18010576b9587f3fc395855804ee1f7e84e3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-web6414t/wheels/41/60/41/81898870259d7c19fe8f9e46a537611c939f0c425eee2e1785
Successfully built de-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_md')


In [0]:
!pip install python-docx
#!pip install python-docx 
#!pip install --upgrade git+https://github.com/flairNLP/flair.git

     |████████████████████████████████| 5.5MB 4.7MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=7e29e7909149e682527f4291f86be652f929433577915a674c588daa198d8a31
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [0]:
import de_core_news_md
import spacy
from spacy import displacy
from spacy.scorer import Scorer

nlp = de_core_news_md.load()
#test doc
doc = nlp(u"Die Boehringer Ingelheim RCV GmbH & Co KG  (nachfolgend  BI RCV ), ein Unternehmer mit Sitz in Österreich, ist im Bereich der  Krebsforschung , der Forschung, Entwicklung und Produktion  biopharmazeutischer Arzneimittel  sowie des  Humanpharma-  und  Tiergesundheitsgeschäfts  tätig.  BI RCV  ist  für den Vertrieb in den Vertriebsgebieten Österreich und CEE verantwortlich.")


# Unzip training files

In [0]:
#@title Default title text
!unzip target.zip
#!unzip source.zip

Archive:  target.zip
   creating: target/
  inflating: target/Doc_ (182).docx.out.txt  
  inflating: target/Doc_ (224).docx.out.txt  
  inflating: target/Doc_ (28).docx.out.txt  
  inflating: target/Doc_ (195).docx.out.txt  
  inflating: target/Doc_ (299).docx.out.txt  
  inflating: target/Doc_ (128).docx.out.txt  
  inflating: target/PwC Memo_Firmenwert.docx.out.txt  
  inflating: target/Doc_ (11).docx.out.txt  
  inflating: target/Doc_ (638).docx.out.txt  
  inflating: target/CCI Valve Technology GmbH_Anforderungsliste_StE 2019.docx.out.txt  
  inflating: target/__tax-dms.eurad.ad.pwcinternal.com__at__SSL_DavWWWRoot_sites_0.0806217.001_Documents_06_BetriebsprÅfung_Korr_MK_BP_Telefonnotiz_23092019.docx.out.txt  
  inflating: target/0423_BI RCV_Memo BS_Clean.docx.out.txt  
  inflating: target/Begleitschreiben_HN.docx.out.txt  
  inflating: target/Doc_ (91).docx.out.txt  
  inflating: target/Doc_ (220).docx.out.txt  
  inflating: target/Doc_ (237).docx.out.txt  
  inflating: target/Doc_

## Reading target files and fill level 1 dict

In [0]:
import os
root = "."
folder = "target"
path = os.path.join(root,folder)
ner_data = []
filePath = [] # data path
labels_list = []

for file_path, dirs, files in os.walk(path):
    for name in files:

            
            # path of the pdf
            if name.split('.')[-1] == 'txt':
              path = os.path.join(file_path[len(root)+1:],name)
              filePath.append(path)
              f = open(path, "r")
              lines = f.readlines()
              #print(path)
              for line in lines:

                label = line.split(' || ')[1]
                txt = line.split(' || ')[0]

                #print(label)
                label = eval(label.replace('(','"').replace(')','"'))
                post_lb_list =[]
                if len(label) > 0 :
                  for lb in label:
                    label_v = lb.split(',')[2]

                    labels_list.append(label_v)

us_spacy_dict = dict()
for lb in labels_list:
  if 'name' in lb.lower():
    us_spacy_dict[lb] = 'PER'
  elif 'location' in lb.lower():
    us_spacy_dict[lb] = 'LOC'
  elif lb == 'CLIENT':
    us_spacy_dict[lb] = 'ORG'
  elif lb == 'CLIENT_SUBSIDARY':
    us_spacy_dict[lb] = 'ORG'
  elif lb == 'CLIENT_SISTER-CO':
    us_spacy_dict[lb] = 'ORG'
  elif lb == 'COMPETITOR_CLIENT':
    us_spacy_dict[lb] = 'ORG'
  else: 
    us_spacy_dict[lb] = ''

#Utility functions

In [0]:
import re


# Correct overlapps (two labels on the same/ovelapped words)
def correct_overlap(s1,s2,e1,e2, en1, en2, txt):
  if s1 <= s2 and e1 >= e2:
    #s1------------e1#
    #----s2-----e2#
      return True,s1,e1,en1

  elif s1 <= s2 and e2 >= e1 and s2 <= e1:
    #s1------------e1#
    #----s2---------------e2#
      return True,s1,e2,txt[s1:e2]

  elif s2 <= s1 and e1 >= e2 and s1 <= e2:
    #s2------------e2#
    #----s1---------------e1#
      return True,s2,e1,txt[s2:e1]
      
  elif s2 <= s1 and e2 >= e1:
    #s2------------e2#
    #----s1-----e1#
    return True,s2,e2,txt[s2:e2]
  else:
    return False,0,0,0


# Correct indexes after the correction of overlaps
def correct_spaces(txt, words):
  
  r = re.compile(re.escape('%s' % words), re.I)
  m = r.search(txt)
  if not(m):
    print('issue',txt, 'words',words)
  index = m.start()
  endx = m.end()
  return index,endx
# test
txt = "REWE International Dienstleistungs GmbH z.H. Mag. (FH) Michael Reithmaier Industriezentrum NÖ-Süd , Straße 3 , Objekt 16 A- 2355 r Neudorf Österreich 4. September 2018 WAJU"
words = "Mag. (FH) Michael"
index,endx = correct_spaces(txt, words)
print(txt[index:endx])


def switch_entity_offsets(entity_offsets):
  # switch between:
  #[{'start': 40, 'end': 53, 'label': 'LOC'}, ..]
  #and -----------------
  #[(40, 52, 'LOC'), ..
  entity_offsets_post = []
  for en in entity_offsets:
    entity_offsets_post.append((en['start'],en['end'],en['label']))
  return entity_offsets_post

# test
entity_offsets = [{'start': 17, 'end': 24, 'label': 'ORG'}, {'start': 26, 'end': 39, 'label': 'PER'}]
entity_offsets_post = switch_entity_offsets(entity_offsets)
entity_offsets_post

def split_tuple(lb):
      # This function is userd to precess the tuple labels and return the label plus the starting and ending index
      label_ch= lb.split(',')[2]  # the label PER, LOC..
      leng = int(lb.split(',')[1])  # number of characters in the labeled text 
      start_ch = int(lb.split(',')[0])  # the index of first charachter 
      end_ch = start_ch + leng  # the index of end charachter 

      return start_ch, end_ch, label_ch

def correct_overlap_loop(post_lb_list, start_ch, end_ch, txt, label_ch):
      # This function is detect ovelaped labeling 
      con_c = False
      for lndi, ldn in enumerate(post_lb_list):
        end_p_t = ldn[1]   # end char
        start_t = ldn[0]  # start char 
        current_text = txt[start_ch:end_ch]
        potential_overlap_text = txt[start_t:end_p_t]

        detected_ov,s1,e1,ent = correct_overlap(start_ch,start_t, end_ch, end_p_t, current_text, potential_overlap_text, txt )

        # If an overlap is detected use the minimum starting index to the maxium index of ovelapped texts
        if detected_ov:
          post_lb_list[lndi] = (s1, e1, us_spacy_dict[label_ch])
          con_c = True

      return post_lb_list, con_c

def  test_proformance_considering_ovelap(post_lb_list, txt_post, txt_orign, num_over, doc ):
      # This function is used to detect overlapps of predicted labels and ground truth labels
      post_space_correction_lb_list =[] 
      for lndi, ldn in enumerate(post_lb_list):
          end_p_t = ldn[1]
          start_t = ldn[0]
          sp, ep = correct_spaces(txt_post,txt_orign[start_t:end_p_t].replace('  ',' ').replace('  ',' '))  # correct the indexes after applying the replace ops
          post_space_correction_lb_list.append((sp,ep,ldn[2]))


          for spe in doc.to_json()['ents']:  # loop over Spacy labels
            sp2 = spe['start']  # starting character
            ep2= spe['end']  # ending character
            
            Is_det,_,_,_ = correct_overlap(sp,sp2,ep,ep2, txt_post[sp:ep], txt_post[sp2:ep2],txt_post) # use the same function above to detect ovelaps between predicted and gt labels

            if Is_det:
              if txt_post[sp2:ep2] not in list_pr:
                list_pr.append(txt_post[sp2:ep2])
                if test_in > 200:
                  num_over+=1
                #print('--------------')
                #print('GT',txt_post[sp:ep])
                #print('PR',txt_post[sp2:ep2])
              
              continue

      return post_space_correction_lb_list, num_over


Mag. (FH) Michael


In [0]:
mammoth.__version__

AttributeError: ignored

In [0]:
import mammoth
with open("/content/Doc_ (152).docx", "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    html = result.value

In [0]:
!pip install docx2html

  Using cached https://files.pythonhosted.org/packages/b3/a9/e943f7b36f1a8afd8b234b9e999be8837ab2666c7e743a4d527e14d39053/docx2html-0.2.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/91/85/2776a729f8ef1e2aca83e02393dbdfcdf2c0f7c5a1205a77bbf1f82230f9/lxml-2.2.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/77/93/d6eabe91dda55ac924c95763568d596e3e6082950a04c24fccf32c12324c/Pillow-1.7.7.zip
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [0]:
!pip install mammoth

In [0]:
!git clone https://github.com/MisterKeefe/python-mammoth.git

Cloning into 'python-mammoth'...
remote: Enumerating objects: 3734, done.
remote: Total 3734 (delta 0), reused 0 (delta 0), pack-reused 3734
Receiving objects: 100% (3734/3734), 777.73 KiB | 16.55 MiB/s, done.
Resolving deltas: 100% (2719/2719), done.


In [0]:
%python 'python-mammoth/setup.py'

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [0]:
#%cd python-mammoth 
!python setup.py install

running install
running bdist_egg
running egg_info
creating mammoth.egg-info
writing mammoth.egg-info/PKG-INFO
writing dependency_links to mammoth.egg-info/dependency_links.txt
writing entry points to mammoth.egg-info/entry_points.txt
writing requirements to mammoth.egg-info/requires.txt
writing top-level names to mammoth.egg-info/top_level.txt
writing manifest file 'mammoth.egg-info/SOURCES.txt'
writing manifest file 'mammoth.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mammoth
copying mammoth/conversion.py -> build/lib/mammoth
copying mammoth/html_paths.py -> build/lib/mammoth
copying mammoth/options.py -> build/lib/mammoth
copying mammoth/images.py -> build/lib/mammoth
copying mammoth/raw_text.py -> build/lib/mammoth
copying mammoth/__init__.py -> build/lib/mammoth
copying mammoth/cli.py -> build/lib/mammoth
copying mammoth/documents.py -> build/lib/mammoth
copy

In [0]:
%cd ../

/content


In [84]:
import docx
import mammoth

doc_name = 'PwC Brief.docx'
doc = docx.Document(doc_name)
for para_id, para in enumerate(doc.paragraphs):
        # make a sentence
        #if len(para.text)>0:
            print(para_id, para.text)
#for para_id, para in enumerate(doc.sections):
        # make a sentence
        #if len(para.text)>0:
 #       print(para_id, para)
f = open(doc_name, 'rb')
#mammoth.convert_to_markdown
document = mammoth.convert_to_html(f)
f.close()

document.value


0 	
1 
2 
3 
4 
5 
6 Steuerpflichtiger: Michael Kors (Austria) GmbH
7 	 Steuernummer 357/4506, Team 23
8 
9 
10 
11 vertreten durch:	PwC PricewaterhouseCoopers
12 	Wirtschaftsprüfung und Steuerberatung GmbH
13 	(Vollmacht erteilt)
14 
15 
16 
17 
18 	MMag. Gerald Dipplinger	i.V. Mag. Nicole Kruckenfellner
19 	  gerald.dipplinger@pwc.com	 nicole.kruckenfellner@pwc.com
20 	01-501 88-3648	01-501 88-3647
21 
22 
23 
24 wegen:	Beilagen zur Körperschaftsteuererklärung 2018
25 
26 
27 
28 
29 
30 
31 
32 EINSCHREIBEN
33 Finanzamt Bruck Eisenstadt Oberwart
34 Team 23

35 Neusiedlerstraße 46
36 7001 Eisenstadt
37 	
38 
39 				
40 19. September 2019
41 
42 
43 
44 Michael Kors (Austria) GmbH
45 Steuernummer 357/4506, Team 23
46 
47 
48 Hiermit übermitteln wir dem Finanzamt die Beilagen I und II zur Körperschaftsteuerklärung für das Jahr 2018 sowie den Jahresabschluss zum 31.3.2018.
49 
50 Die Körperschaftsteuererklärung für das Jahr 2018 wurde am 31. Juli 2019 per FinanzOnline
51 eingereic

'<p><a id="Betreff"></a>\t</p><p><strong>EINSCHREIBEN</strong></p><p>Finanzamt Bruck Eisenstadt Oberwart</p><p><a id="OLE_LINK2"></a><a id="OLE_LINK3"></a>Team 23<br /></p><p>Neusiedlerstraße 46</p><p><strong>7001 Eisenstadt</strong></p><p>19. September 2019</p><p>KOJL</p><p><strong>Steuerpflichtiger: </strong>Michael Kors (Austria) GmbH</p><p>\t Steuernummer 357/4506, Team 23</p><p>vertreten durch:\tPwC PricewaterhouseCoopers</p><p>\tWirtschaftsprüfung und Steuerberatung GmbH</p><p>\t(Vollmacht erteilt)</p><p>\tMMag. Gerald Dipplinger\ti.V. Mag. Nicole Kruckenfellner</p><p>\t\uf02a  gerald.dipplinger@pwc.com\t\uf02a nicole.kruckenfellner@pwc.com</p><p>\t\uf028\uf02001-501 88-3648\t\uf028\uf02001-501 88-3647</p><p><strong>wegen:</strong>\t<strong>Beilagen zur Körperschaftsteuererklärung 2018</strong></p><p><strong>EINSCHREIBEN</strong></p><p>Finanzamt Bruck Eisenstadt Oberwart</p><p>Team 23<br /></p><p>Neusiedlerstraße 46</p><p><strong>7001 Eisenstadt</strong></p><p>\t</p><p>\t\t\t\t</

In [0]:
doc.sections

In [0]:
!pip install beautifulsoup4

In [0]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(document.value, 'lxml')

In [96]:
for par in soup.findAll('p'):
  

'19. September 2019'

In [0]:
import IPython
IPython.display.HTML(document.value)


In [0]:

	
	ProcessLookupErro




IndentationError: ignored

# Text to structured data 

In [0]:
#!unzip target.zip
import os
root = "."
folder = "target"
path = os.path.join(root,folder)
ner_data = []
filePath = [] # data path
labels_list = []
post_labels_list = []
con_c = False
num_ent_po = 0
num_ent_gr = 0
num_over = 0
f1 = []
optimizer = nlp.begin_training()
test_in = 189
for file_path, dirs, files in os.walk(path):

    for name in files[190:]:
      test_in+=1
      print('-------------------------------------------------------------------', test_in)
      if  True:
            print("filename---",name)

            # path of the pdf
            if name.split('.')[-1] == 'txt':

              path = os.path.join(file_path[len(root)+1:],name)
              filePath.append(path)
              f = open(path, "r")
              lines = f.readlines()
              #print(path)
              for line in lines:

                label = line.split(' || ')[1] # labels part of the training data
                txt_origin = line.split(' || ')[0]  # text part of the training data

                #print("text---",txt_origin)
                label = eval(label.replace('(','"').replace(')','"')) # replace brackets with ""
                post_lb_list =[]  # initialize the labels list of current paragraph/line
                if len(label) > 0 :  # if there are ground truth labels
                  for lb in label:
                    con_c = False # a boolean variable to exit the loop once an overlapp is detected (same text is labeled twice)
                    # lb 0,5,LOCATION_RECIPIENT_ZIPCODE --- start char index, number of chars, label
                    
                    start_ch, end_ch, label_ch = split_tuple(lb)

                    # Correct overlapps (two labels on the same/ovelapped words)
                    post_lb_list,con_c = correct_overlap_loop(post_lb_list, start_ch, end_ch, txt_origin, label_ch)

                    # If an ovelap was detected ignore the rest
                    if con_c:
                      continue  

                    # update labels list of current paragraph/line
                    if us_spacy_dict[label_ch] != '':
                      post_lb_list.append((start_ch, end_ch, us_spacy_dict[label_ch])) # level 1-3 labels
                      post_labels_list.append(us_spacy_dict[label_ch]) # map all levels to level 1

                  # Correct extra spaces caused by parsing docx files
                  txt_post = txt_origin.replace('  ',' ').replace('  ',' ') 
                  list_pr = []

                  # Run spacy ner
                  doc = nlp(txt_post)

                  if test_in > 200:  # start testing index, before 200 do training

                    num_ent_po+=len(doc.to_json()['ents'])  # number of predicted labels
                    num_ent_gr +=len(list(dict.fromkeys(post_lb_list)))  # number of original labels


                  post_space_correction_lb_list, num_over = test_proformance_considering_ovelap(post_lb_list, txt_post, txt_origin, num_over, doc )
                          
                  post_lb_list = list(dict.fromkeys(post_space_correction_lb_list))
                  #print(list(dict.fromkeys(post_space_correction_lb_list)))

                  ner_data.append((txt.replace('  ',' ').replace('  ',' '),{"entities":post_lb_list}))
                  entity_offsets= post_lb_list
                  #print('entinty_GT', entity_offsets) 
                  try:
                    gold = spacy.gold.GoldParse(doc, entities=entity_offsets)
                    gold2 = spacy.gold.GoldParse(doc, entities=switch_entity_offsets(doc.to_json()['ents']))
                  except:
                    continue
                  scorer = Scorer()
                  scorer.score(doc, gold)
                  #print(entity_offsets)
                  #print(doc)
                  if test_in <= 200:
                    try:
                      nlp.update([doc.text], [gold], drop=0.5, sgd=optimizer)
                    except:
                      continue
                  else:
                        print('GT', entity_offsets)
                        print('Switch', switch_entity_offsets(doc.to_json()['ents']))
                        print('f score',scorer.ents_f) 
                        f1_sc = scorer.ents_f
                        scorer = Scorer()

                        scorer.score(doc,gold2)
                        print('f score', scorer.ents_f) 
                        if len(entity_offsets)> 0:
                          f1.append(f1_sc)
                        print(entity_offsets)
                        print(doc)

                        displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})
                        print("-----------------------------------------")

------------------------------------------------------------------- 190
filename--- Doc_ (64).docx.out.txt
------------------------------------------------------------------- 191
filename--- Doc_ (290).docx.out.txt
------------------------------------------------------------------- 192
filename--- Doc_ (53).docx.out.txt
------------------------------------------------------------------- 193
filename--- Doc_ (396).docx.out.txt
------------------------------------------------------------------- 194
filename--- Doc_ (40).docx.out.txt
------------------------------------------------------------------- 195
filename--- Doc_ (313).docx.out.txt
------------------------------------------------------------------- 196
filename--- Doc_ (320).docx.out.txt
------------------------------------------------------------------- 197
filename--- Doc_ (63).docx.out.txt
------------------------------------------------------------------- 198
filename--- Doc_ (143).docx.out.txt
--------------------------------

-----------------------------------------
GT [(10, 20, 'LOC')]
Switch [(10, 20, 'LOC')]
f score 100.0
f score 100.0
[(10, 20, 'LOC')]
Finanzamt Graz-Stadt 


-----------------------------------------
GT [(0, 28, 'LOC'), (30, 35, 'LOC')]
Switch [(0, 28, 'LOC'), (30, 35, 'LOC')]
f score 100.0
f score 100.0
[(0, 28, 'LOC'), (30, 35, 'LOC')]
Conrad von Hötzendorf-Straße  14-18 


-----------------------------------------
GT [(0, 5, 'LOC'), (6, 10, 'LOC')]
Switch [(0, 4, 'LOC'), (6, 10, 'LOC')]
f score 0.0
f score 100.0
[(0, 5, 'LOC'), (6, 10, 'LOC')]
8010  Graz 


-----------------------------------------
GT [(0, 4, 'PER')]
Switch [(0, 4, 'PER')]
f score 100.0
f score 100.0
[(0, 4, 'PER')]
hevn


...

-----------------------------------------
GT [(18, 24, 'PER')]
Switch []
f score 0.0
f score 0.0
[(18, 24, 'PER')]
Sehr geehrte Frau Anders ,


/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


-----------------------------------------
GT [(0, 35, 'ORG')]
Switch [(0, 35, 'ORG'), (148, 183, 'ORG')]
f score 66.66666666666666
f score 100.0
[(0, 35, 'ORG')]
Douwe Egberts Coffee Tr & Supply BV führt seit Juni 201 3 keine steuerbaren Umsätze in Österreich mehr aus. Da eine steuerliche Deregistrierung der Douwe Egberts Coffee Tr & Supply BV jedoch erst nach Abgabe der Umsatzsteuerjahreserklärung 201 3 möglich ist, ersuchen wir aus Gründen der administrativen Vereinfachung von der Verpflichtung zur Einreichung von Null- Umsatzsteuervoranmeldung für den Zeitraum ab Juni 201 3 abzusehen.


-----------------------------------------
GT [(0, 1, 'ORG')]
Switch [(0, 1, 'PER')]
f score 0.0
f score 100.0
[(0, 1, 'ORG')]
D


-----------------------------------------
------------------------------------------------------------------- 271
filename--- __tax-dms.eurad.ad.pwcinternal.com__at__SSL_DavWWWRoot_sites_0.0861988.001_Documents_04_Compliance_2018_Jahresabschluss_Tel.Notiz_Hr._Peters_29042019.docx.out.txt


In [0]:
import numpy as np
np.mean(f1)

56.55248172605289

In [0]:
nlp.to_disk("/ner_lv1_ model")

In [0]:
raw_text = "EINSCHREIBEN Finanzamt Salzburg-Stadt Team 25 Aigner Straße  10 5026 Salzburg 27. Mai 2020 opan"
text[38:45]
doc = nlp.make_doc(raw_text)
gold = spacy.gold.GoldParse(doc, entities=entity_offsets)
for i in range(100):
  nlp.update([doc], [gold], drop=1, sgd=optimizer)

In [0]:
raw_text = "EINSCHREIBEN Finanzamt Salzburg-Stadt Team 25 Aigner Straße  10 5026 Salzburg 27. Mai 2020 opan"

doc = nlp(raw_text)

displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

In [0]:
num_over/num_ent_po

0.9396919042936742

In [0]:
num_over/num_ent_po

0.9458393113342898

In [0]:
html = displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

In [0]:
html